In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TextStreamer, GenerationConfig

model_name='davidkim205/komt-mistral-7b-v1'
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16, low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer)

def gen(x):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.8,
        top_k=100,
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
    )
    q = f"[INST]{x} [/INST]"
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"\n\n### Response: "
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [2]:
import platform
import os
import pandas as pd


TEST_NAME = "llm1"

# 데이터 관련
os_name = platform.system()
if os_name == 'Windows':
    PRE_PATH = ''
elif os_name == 'Linux':
    PRE_PATH = '/kkh/'
elif os_name == 'Darwin': # 맥
    PRE_PATH = '/kkh/'
DATA_PATH = PRE_PATH + "data/" # 대회에서 제공한 데이터
OUTPUT_PATH = PRE_PATH + "output/" # 모델의 출력 값
CHECKPOINT_PATH = PRE_PATH + "checkpoint/" # 모델의 최종 출력 값
PREDICTION_PATH = PRE_PATH + "prediction/" # 최종 예측 값
LOG_PATH = PRE_PATH + "log/"
TRAIN_PATH = DATA_PATH + "train_new2_temp.csv"
VALID_PATH = DATA_PATH + "dev_new2.csv"
TEST_PATH = DATA_PATH + "test.csv"
SUBMIT_PATH = PREDICTION_PATH + "submission_" + TEST_NAME + ".csv"

train_df = pd.read_csv(os.path.join(TRAIN_PATH))
val_df = pd.read_csv(os.path.join(VALID_PATH))
test_df = pd.read_csv(os.path.join(TEST_PATH))

sampled_df = train_df.sample(n=5, random_state=42)  # random_state는 결과를 재현 가능하게 합니다.
combined_string = '\n'.join(sampled_df.apply(lambda row: f"{row['fname']}, {row['dialogue']}, {row['summary']}, {row['topic']}", axis=1).tolist())
print(combined_string)

train_1057, #Person1#: 그래도 저는 이 벽지를 원하지 않았어요. 그리고 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요. 
#Person2#: 그 끔찍한 바이러스에 대해 들었어요! 자동으로 주소록에 있는 모든 사람에게 첨부 파일로 이메일을 보내요.
#Person1#: 글쎄요, 이미 제 컴퓨터에 올라와 있어요. 어떻게 없앨 수 있죠?
#Person3#: 다음 선거에서 투표로 없애세요.
#Person1#: 이제 그만하고 진지하게 대답해줘요!
#Person3#: IT 부서에게 바이러스 제거 프로그램을 실행하도록 요청하세요. 2005년까지 문제가 해결될 것이라고 생각해요., #Person1#의 컴퓨터에 끔찍한 바이러스가 걸렸고, #Person2#는 IT 부서에게 바이러스 제거 프로그램을 실행하도록 제안했다., 컴퓨터 바이러스
train_10667, #Person1#: 우리 회사에 관심이 있는 이유는 무엇인가요?
#Person2#: 귀사는 세계적으로 유명합니다. 그래서 개인적인 발전을 위한 더 나은 기회가 있을 것이라고 생각합니다.
#Person1#: 맞습니다. 이 직무에 필요한 자격을 갖추고 있다고 생각하나요?
#Person2#: 그럼요. 제 자격은 귀사의 직무 설명에 부합합니다.
#Person1#: 하지만 이 직무에 대한 경험이 없습니다.
#Person2#: 그건 사실입니다. 하지만 배우는 것에 열정이 있고, 빠르게 배울 수 있습니다.
#Person1#: 추가 교육이 필요하다면, 교육비를 아시나요?
#Person2#: 아니요. 월급은 얼마인가요?
#Person1#: 월 800위안입니다.
#Person2#: 내일 답변을 드려도 괜찮을까요?
#Person1#: 예. 물론입니다., #Person2#는 세계적으로 유명한 #Person1#의 회사에 관심이 있으며, 경험이 없음에도 불구하고 #Person2#는 자신이 직무에 필요한 자격을 갖추고 있다고 생각합니다., 직무 인터뷰
train_5261, #Person1#: 데이비드, 우리가 여기 앉아

In [7]:

for index, row in test_df.iterrows():
    print(f"대화문: {row['fname']}")
    print(row['dialogue'])
    row['summary'] = '"' + gen(f"{combined_string} \n\n\n 위와 같은 대화문과 요약문이 존재하니, 참고해줘.\n\
    아래 대화문에서 핵심 키워드를 3개 정도 추출해서 출력해주고, 핵심 키워드를 꼭 포함해서 요약문을 작성해줘.\n\
    요약문은 한 문장 또는 두 문장으로 표현해주고, 더불어, 요약문의 len() 길이는 101을 넘지 않도록 요약해줘.\n\
    그리고, 내가 적었던 문장을 다시 출력하지 말아줘.\n\
    \n{row['dialogue']}") + '"'
    print(row['summary'])
    print('=====================================')

대화문: train_1
#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?
#Person2#: 안녕하세요, 피터스 박사님. 잘 지냈습니다, 감사합니다. 리키와 함께 백신 접종을 위해 왔습니다.
#Person1#: 좋습니다. 백신 접종 기록을 보니, 리키는 이미 소아마비, 디프테리아, B형 간염 백신을 맞았군요. 그는 14개월이므로, 이제 A형 간염, 수두, 홍역 백신을 맞아야 합니다.
#Person2#: 풍진과 볼거리는 어떻게 되나요?
#Person1#: 지금은 이 백신들만 접종할 수 있고, 몇 주 후에 나머지를 접종할 수 있습니다.
#Person2#: 좋습니다. 박사님, 저도 디프테리아 예방접종이 필요할 것 같아요. 마지막으로 맞은 게 아마도 15년 전이었던 것 같아요!
#Person1#: 저희가 기록을 확인하고 간호사에게 부스터를 접종하도록 하겠습니다. 이제, 리키의 팔을 꽉 잡아주세요, 조금 찌릿할 수 있습니다.
<s> [INST]train_1057, #Person1#: 그래도 저는 이 벽지를 원하지 않았어요. 그리고 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요. 
#Person2#: 그 끔찍한 바이러스에 대해 들었어요! 자동으로 주소록에 있는 모든 사람에게 첨부 파일로 이메일을 보내요.
#Person1#: 글쎄요, 이미 제 컴퓨터에 올라와 있어요. 어떻게 없앨 수 있죠?
#Person3#: 다음 선거에서 투표로 없애세요.
#Person1#: 이제 그만하고 진지하게 대답해줘요!
#Person3#: IT 부서에게 바이러스 제거 프로그램을 실행하도록 요청하세요. 2005년까지 문제가 해결될 것이라고 생각해요., #Person1#의 컴퓨터에 끔찍한 바이러스가 걸렸고, #Person2#는 IT 부서에게 바이러스 제거 프로그램을 실행하도록 제안했다., 컴퓨터 바이러스
train_10667, #Person1#: 우리 회사에 관심이 있는 이유는 무엇인가요?
#Person2#: 귀사는 세계적으로 유명합니다. 그래서 개인적인 발전을 위한 더 

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 738.00 MiB. GPU 